In [ ]:
!pip install openai

from openai import OpenAI
from google.colab import userdata


# OpenAI API 클라이언트 생성
API_KEY = userdata.get('api_key')

Few-shot : 단 몇 개의 예시(examples)를 프롬프트(지시문)에 포함하여 제공하는 방법(In-context Learning)

| 개념              | 설명                                                                 | 예시 (감정 분석) |
|-------------------|----------------------------------------------------------------------|------------------|
| **Zero-shot (제로샷)** | 예시 없이, 오직 지시만으로 작업을 요청. 모델의 기존 지식에 전적으로 의존. | "이 문장은 긍정이야 부정이야?" |
| **One-shot (원샷)**    | 단 하나의 예시를 제공. 원하는 출력 형식을 명확히 할 때 유용.            | 입력: "배송 빨라요." -> 긍정<br>입력: "별로네요." -> ? |
| **Few-shot (퓨샷)**    | 몇 개(2~5개)의 예시를 제공. 더 복잡한 패턴이나 뉘앙스를 학습시키기에 효과적. | (위의 구체적인 예시와 동일) |
| **Fine-tuning (파인튜닝)** | 수백~수천 개 이상의 많은 데이터를 사용해 모델의 가중치(weights)를 직접 업데이트. 모델을 특정 작업에 깊게 전문화시키는 과정. | 감정분석 데이터셋으로 모델을 재훈련 |


In [ ]:
class PromptEngineering:
    def __init__(self):
        self.client = OpenAI(api_key=API_KEY)  # <- 본인 API 키 입력

    def zero_shot(self, text):
        """Zero-shot 분류 예제"""
        prompt = f"""
        다음 텍스트의 감정을 분석하세요.

        텍스트: {text}
        감정 (긍정/부정/중립):
        """
        return self._call_api(prompt)

    def few_shot(self, text):
        """Few-shot 학습 예제"""
        prompt = f"""
        다음 예시를 참고하여 텍스트를 분류하세요.

        예시 1: "오늘 날씨가 정말 좋네요!" → 긍정
        예시 2: "서비스가 너무 실망스러웠어요." → 부정
        예시 3: "평범한 하루였습니다." → 중립

        텍스트: {text}
        분류:
        """
        return self._call_api(prompt)

    def chain_of_thought(self, problem):
        """Chain of Thought 추론 예제"""
        prompt = f"""
        다음 문제를 단계별로 풀어주세요.

        문제: {problem}

        풀이 과정:
        1단계: 문제 이해
        2단계: 필요한 정보 파악
        3단계: 계산 과정
        4단계: 최종 답
        """
        return self._call_api(prompt)

    def _call_api(self, prompt):
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        return response.choices[0].message.content

# 테스트
pe = PromptEngineering()

print(" Zero-shot 결과:")
print(pe.zero_shot("이 제품 정말 마음에 들어요!"))
print("\n" + "="*50 + "\n")

print(" Few-shot 결과:")
print(pe.few_shot("가격대비 괜찮은 것 같아요"))
print("\n" + "="*50 + "\n")

print(" Chain of Thought 결과:")
print(pe.chain_of_thought("사과 5개와 배 3개가 있습니다. 친구에게 사과 2개를 주고, 배 1개를 먹었다면 남은 과일은 몇 개인가요?"))

 Zero-shot 결과:
감정: 긍정


 Few-shot 결과:
긍정


 Chain of Thought 결과:
문제를 단계별로 풀어보겠습니다.

### 1단계: 문제 이해
사과와 배가 주어져 있으며, 사과를 친구에게 주고 배를 먹는 상황입니다. 남은 과일의 개수를 구하는 것이 목표입니다.

### 2단계: 필요한 정보 파악
- 처음 사과의 개수: 5개
- 처음 배의 개수: 3개
- 친구에게 준 사과의 개수: 2개
- 먹은 배의 개수: 1개

### 3단계: 계산 과정
1. 사과의 남은 개수 계산:
   - 처음 사과: 5개
   - 친구에게 준 사과: 2개
   - 남은 사과 = 5 - 2 = 3개

2. 배의 남은 개수 계산:
   - 처음 배: 3개
   - 먹은 배: 1개
   - 남은 배 = 3 - 1 = 2개

3. 남은 과일의 총 개수 계산:
   - 남은 사과: 3개
   - 남은 배: 2개
   - 남은 과일의 총 개수 = 3 + 2 = 5개

### 4단계: 최종 답
남은 과일은 총 5개입니다.
